In [6]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 11: Corporate Policy ---</h2><p>Santa's previous password expired, and he needs help choosing a new one.</p>
<p>To help him remember his new password after the old one expires, Santa has devised a method of coming up with a password based on the previous one.  Corporate policy dictates that passwords must be exactly eight lowercase letters (for security reasons), so he finds his new password by <em>incrementing</em> his old password string repeatedly until it is valid.</p>
<p>Incrementing is just like counting with numbers: <code>xx</code>, <code>xy</code>, <code>xz</code>, <code>ya</code>, <code>yb</code>, and so on. Increase the rightmost letter one step; if it was <code>z</code>, it wraps around to <code>a</code>, and repeat with the next letter to the left until one doesn't wrap around.</p>
<p>Unfortunately for Santa, a new Security-Elf recently started, and he has imposed some additional password requirements:</p>
<ul>
<li>Passwords must include one increasing straight of at least three letters, like <code>abc</code>, <code>bcd</code>, <code>cde</code>, and so on, up to <code>xyz</code>. They cannot skip letters; <code>abd</code> doesn't count.</li>
<li>Passwords may not contain the letters <code>i</code>, <code>o</code>, or <code>l</code>, as these letters can be mistaken for other characters and are therefore confusing.</li>
<li>Passwords must contain at least two different, non-overlapping pairs of letters, like <code>aa</code>, <code>bb</code>, or <code>zz</code>.</li>
</ul>
<p>For example:</p>
<ul>
<li><code>hijklmmn</code> meets the first requirement (because it contains the straight <code>hij</code>) but fails the second requirement requirement (because it contains <code>i</code> and <code>l</code>).</li>
<li><code>abbceffg</code> meets the third requirement (because it repeats <code>bb</code> and <code>ff</code>) but fails the first requirement.</li>
<li><code>abbcegjk</code> fails the third requirement, because it only has one double letter (<code>bb</code>).</li>
<li>The next password after <code>abcdefgh</code> is <code>abcdffaa</code>.</li>
<li>The next password after <code>ghijklmn</code> is <code>ghjaabcc</code>, because you eventually skip all the passwords that start with <code>ghi...</code>, since <code>i</code> is not allowed.</li>
</ul>
<p>Given Santa's current password (your puzzle input), what should his <em>next password</em> be?</p>
</article>
<p>Your puzzle input is <code class="puzzle-input">hepxcrrq</code>.</p>


In [7]:
valid_tests = [
    {
        "name": "Example 1",
        "password": "hijklmmn",
        "expected": False,
    },
    {
        "name": "Example 2",
        "password": "abbceffg",
        "expected": False,
    },
    {
        "name": "Example 3",
        "password": "abbcegjk",
        "expected": False,
    },
    {
        "name": "Example 4",
        "password": "abcdefgh",
        "expected": False,
    },
    {
        "name": "Example 5",
        "password": "abcdffaa",
        "expected": True,
    },
    {
        "name": "Example 6",
        "password": "ghijklmn",
        "expected": False,
    },
    {
        "name": "Example 7",
        "password": "ghjaabcc",
        "expected": True,
    },
    {
        "name": "Example 8",
        "password": "hepxcrrq",
        "expected": False,
    },
]


def valid(password: str) -> bool:
    # passwords must be exactly eight lowercase letters (for security reasons),
    if len(password) != 8:
        return False
    # Passwords must include one increasing straight of at least three letters,
    # like abc, bcd, cde, and so on, up to xyz.
    if all(
        ord(password[i]) - ord(password[i - 1]) != 1
        or ord(password[i - 1]) - ord(password[i - 2]) != 1
        for i in range(2, len(password))
    ):
        return False
    # Passwords may not contain the letters i, o, or l
    if re.search(r"[i|o|l]+", password):
        return False
    # Passwords must contain at least two different, non-overlapping pairs of letters,
    # like aa, bb, or zz.
    return len(re.findall(r"(.)\1", password)) > 1


run_tests_params(valid, valid_tests)


Test Example 1 passed, for valid.
Test Example 2 passed, for valid.
Test Example 3 passed, for valid.
Test Example 4 passed, for valid.
Test Example 5 passed, for valid.
Test Example 6 passed, for valid.
Test Example 7 passed, for valid.
Test Example 8 passed, for valid.
Success


In [8]:
from functools import cache
from string import ascii_lowercase

from more_itertools import first, take

valid_tests = [
    {
        "name": "Example 1",
        "password": "abcdefgh",
        "expected": "abcdffaa",
    },
    {
        "name": "Example 2",
        "password": "ghijklmn",
        "expected": "ghjaabcc",
    },
]


def next_valid(password: str) -> str:
    def nxt(acc: str, index: int, start=False) -> str:
        if index == len(password):
            yield acc
        else:
            j = ascii_lowercase.index(password[index]) + 1 if start else 0
            for i in range(j, len(ascii_lowercase)):
                if not ascii_lowercase[i] in "iol":
                    yield from nxt(acc + ascii_lowercase[i], index + 1)

    def start():
        for index in range(len(password) - 1, -1, -1):
            acc = password[:index]
            if not re.search(r"[i|o|l]+", acc):
                yield from nxt(acc, index, True)

    chars = "abcdefghjkmnpqrstuvwxyz"
    return first(pw for pw in start() if valid(pw))


run_tests_params(next_valid, valid_tests)


Test Example 1 passed, for next_valid.
Test Example 2 passed, for next_valid.
Success


In [9]:
next_valid("hepxcrrq")

'hepxxyzz'

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>hepxxyzz</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Santa's password <span title="The corporate policy says your password expires after 12 seconds.  For security.">expired again</span>.  What's the next one?</p>
</article>

</main>


In [10]:
next_valid("hepxxyzz")

'heqaabcc'

<link href="style.css" rel="stylesheet"></link>

<main>

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Santa's password <span title="The corporate policy says your password expires after 12 seconds.  For security.">expired again</span>.  What's the next one?</p>
</article>
<p>Your puzzle answer was <code>heqaabcc</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>Your puzzle input was <code class="puzzle-input">hepxcrrq</code>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Corporate+Policy%22+%2D+Day+11+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F11&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Corporate+Policy%22+%2D+Day+11+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F11';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
